In [1]:
import time
import pandas as pd
import numpy as np
import sklearn
#print(sklearn.__version__)
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
%matplotlib inline
import os
import glob

#using glob to iterate over a folder
files = glob.glob('/data/Clean/Rt-d2/*.csv')
#exist = glob.glob(*)
for fp in files:
    dfs = pd.read_csv(fp,delimiter=',')    
    dfs.columns = ["service_date", "trip_id", "prog_no", "stop_id", "lt_planned_arr", "lt_planned_dep", "lt_actual_arr", "lt_actual_dep", "vehicle_id", "hour", "day", "line_id", "route_id", "direction", "tr_planned_arr", "tr_planned_dep", "tr_actual_arr", "tr_actual_dep", "tr_travel_time", "temp", "wind", "weather"]
    dfs['weather'].replace(np.nan,'Clouds', inplace=True)
    df_39A_prog1 = dfs.loc[dfs.prog_no==1]
    
    #find most frequent stop_id for prog_no== 1 and create a new df
    origin = df_39A_prog1['stop_id'].value_counts().argmax()
    df_mostFrequentOrigin = df_39A_prog1[df_39A_prog1.stop_id == origin]

    #create a new df without prog_no==1
    df_withNoProg1 = dfs[dfs.prog_no != 1]
    
    #concat the 2 dfs 
    df_modeling = pd.concat([df_mostFrequentOrigin, df_withNoProg1], ignore_index=True)
    
    df_modeling['travelTime_from_start'] = df_modeling['lt_actual_arr'] - df_modeling['tr_actual_dep']
    checking2 = df_modeling.loc[df_modeling.tr_actual_dep != 0]
    df = checking2.loc[checking2.travelTime_from_start > 0]
    
    #create binary value for day and time
    df['wkend'] = 0
    df['wkday'] = 0
    df['peakTime'] = 0
    df['offPeakTime'] = 0
    df.loc[df['day'].isin(['Monday','Tuesday','Wednesday','Thursday','Friday']),'wkday'] = 1
    df.loc[df['day'].isin(['Saturday','Sunday']),'wkend'] = 1
    df.loc[df['hour'].isin([7,8,9,10,16,17,18,19]),'peakTime'] = 1
    df.loc[df['hour'].isin([0,1,2,3,4,5,6,11,12,13,14,15,20,21,22,23]),'offPeakTime'] = 1
    df.loc[df['day'].isin(['Saturday','Sunday']),'peakTime'] = 0
    df.loc[df['day'].isin(['Saturday','Sunday']),'oofPeakTime'] = 1
    
    # Prepare the data, turn object feature into dummies.
    df_dummies_weather_main = pd.get_dummies(df[['weather']])
    
    # Prepare the data, turn object feature into dummies. 
    df_dummies_day = pd.get_dummies(df[['day']])
    
    # Prepare the data, turn object feature into dummies. First change to object type
    df[['stop_id']] = df[['stop_id']].astype(object)
    df_dummies_stop_id = pd.get_dummies(df[['stop_id']])
    
    #concat df and generated dummies
    df1 =pd.concat([df[['travelTime_from_start','lt_planned_arr', 'wind', 'temp','peakTime']], df_dummies_weather_main[['weather_Rain']], df_dummies_day, df_dummies_stop_id], axis =1)

    #start modeling
    df2 = shuffle(df1)
    df3 = df2.reset_index()
    
    y = df3['travelTime_from_start']
    X = df3.drop(['travelTime_from_start', 'index'], axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    import sklearn 
    from sklearn.neural_network import MLPRegressor
    #Multi-layer Perceptron is sensitive to feature scaling, so it is highly recommended to scale the data. 

    from sklearn.preprocessing import StandardScaler 
    scaler = StandardScaler()
    SavedScaler = scaler.fit(X_train)  
    X_train_ = scaler.transform(X_train)  
    # apply same transformation to test data
    X_test_ = scaler.transform(X_test) 
    #save scaler to use on the new user input/data
    import pickle as pkl
    a = fp.split('/')[4]
    n = a.split('-')[0]
    if 'd1' not in fp:
        s = 'Scaler' + n + 'I' + '.pkl'
    else:
        s = 'Scaler' + n + 'O' + '.pkl'
    print(s + ' for saving scaler')
    pkl.dump(SavedScaler, open(s, 'wb'))
    
    mlp = MLPRegressor(solver='lbfgs', alpha=1000, hidden_layer_sizes=(20,), learning_rate = 'adaptive', random_state=1)
    model = mlp.fit(X_train_, y_train.values.ravel())
    mlp_res_test = mlp.predict(X_test_)
    
    mape_mlp_test = ((abs((y_test - mlp_res_test)/y_test)) * 100).mean()
    print("\nMean Absolute Percentage Error on train:\n", mape_mlp_test)
    
    from sklearn.metrics import r2_score
    R2_Test = r2_score(y_test, mlp_res_test)
    print("\nANN_R2 Score is:\n", R2_Test)
    
    import pickle as pkl
    a = fp.split('/')[4]
    n = a.split('-')[0]
    if 'd1' not in fp:
        s = n + '_I' + '.pkl'
    else:
        s = n + '_O' + '.pkl'
    print(s)
    pkl.dump(mlp, open(s, 'wb'))
    print('done')
    
    f = open("result.csv", "a")
    f.write(a + ',' + (str(origin)) + ',' + (str(round(mape_mlp_test,2))) + ',' + (str(round(R2_Test,2))) + '\n')
    f.close()

/home/student/anaconda3/envs/Pace/lib/python3.6/site-packages/ipykernel_launcher.py:25: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
/home/student/anaconda3/envs/Pace/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/student/anaconda3/envs/Pace/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#i

Scaler79I.pkl for saving scaler

Mean Absolute Percentage Error on train:
 13.2680866191

ANN_R2 Score is:
 0.924819541289
79_I.pkl
done
Scaler76I.pkl for saving scaler

Mean Absolute Percentage Error on train:
 12.8236740691

ANN_R2 Score is:
 0.910737527551
76_I.pkl
done
Scaler63I.pkl for saving scaler

Mean Absolute Percentage Error on train:
 16.207225157

ANN_R2 Score is:
 0.874732957489
63_I.pkl
done
Scaler61I.pkl for saving scaler

Mean Absolute Percentage Error on train:
 25.840508129

ANN_R2 Score is:
 0.866616605198
61_I.pkl
done
Scaler120I.pkl for saving scaler

Mean Absolute Percentage Error on train:
 12.1032882702

ANN_R2 Score is:
 0.907205782652
120_I.pkl
done
Scaler239I.pkl for saving scaler

Mean Absolute Percentage Error on train:
 120.368319559

ANN_R2 Score is:
 0.867966585638
239_I.pkl
done
Scaler42I.pkl for saving scaler

Mean Absolute Percentage Error on train:
 13.3735395596

ANN_R2 Score is:
 0.915274266426
42_I.pkl
done
Scaler40I.pkl for saving scaler

Mean A